In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Number of training examples
n_train = len(x_train) 

# Number of testing examples.
n_test = len(x_test) 

# Shape of the images
image_shape = x_train[0].shape 

# How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 50000
Number of testing examples = 10000
Image data shape = (32, 32, 3)
Number of classes = 10
<class 'numpy.ndarray'>
(50000, 10)
(10000, 10)


In [3]:
from random import shuffle
from PIL import Image

lr = tf.Variable(0.1) # learning rate
MOMENTUM = 0.9
DECAY = 5e-4 
K=10

num_examples = len(x_train)
num_test = len(x_test)
x = tf.placeholder(tf.float32, shape=(None,)+image_shape)
y = tf.placeholder(tf.int32)
Y = tf.placeholder(tf.float32, shape=(None,K))

def normalize(img,img2):
    img_array = np.asarray(img)
    img_array2 = np.asarray(img2)
    normalized = np.empty(np.shape(img_array))
    normalized2 = np.empty(np.shape(img_array2))
    for i in range(3):
        normalized = (img_array - img_array[:,:,:,i].mean()) / (img_array[:,:,:,i].std())
        normalized2 = (img_array2 - img_array[:,:,:,i].mean()) / (img_array[:,:,:,i].std())
    return normalized,normalized2

x_train,x_test=normalize(x_train,x_test)

def get_one_hot(y,K):
    one_hot=np.zeros((np.shape(y)[0],K))
    for j in range(np.shape(y)[0]):
        one_hot[j,y[j]]=1
    return one_hot

def categorical_loss(F,Y):
    return -tf.math.reduce_mean(tf.math.log(tf.reduce_sum(F*Y,1)),0)

class LossHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))
        print('lr:', step_decay(len(self.losses)))
        
def step_decay(epoch):
    initial_lrate = 0.1
    if epoch<35:
        lrate = initial_lrate
    elif epoch<70:
        lrate = initial_lrate/5
    elif epoch<85:
        lrate = initial_lrate/25
    else:
        lrate = initial_lrate/(25*5)
    return lrate

loss_history = LossHistory()
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
callbacks_list = [loss_history, lrate]

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess: 
    sess.run(tf.global_variables_initializer())
    labels=get_one_hot(y_train,K)
    labels_test=get_one_hot(y_test,K)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
model=tf.keras.models.Sequential()

my_init_1=tf.keras.initializers.RandomNormal(mean=0.0, stddev=tf.math.sqrt(2.0/(5*5*192)))
my_init_2=tf.keras.initializers.RandomNormal(mean=0.0, stddev=tf.math.sqrt(2.0/160))
my_init_3=tf.keras.initializers.RandomNormal(mean=0.0, stddev=tf.math.sqrt(2.0/96))
my_init_4=tf.keras.initializers.RandomNormal(mean=0.0, stddev=tf.math.sqrt(2.0/192))
my_init_5=tf.keras.initializers.RandomNormal(mean=0.0, stddev=tf.math.sqrt(2.0/(3*3*192)))
my_init_dense=tf.keras.initializers.VarianceScaling(scale=2.0, mode='fan_out', distribution='uniform')

model.add(tf.keras.layers.Conv2D(192,kernel_size=5,strides=[1,1],kernel_initializer=my_init_1,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False , input_shape=(32,32,3), padding='same',data_format='channels_last',trainable = False))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05,trainable = False))
model.add(tf.keras.layers.ReLU(trainable = False))
model.add(tf.keras.layers.Conv2D(160,kernel_size=1,strides=[1,1], kernel_initializer=my_init_2, kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same',trainable = False))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05,trainable = False))
model.add(tf.keras.layers.ReLU(trainable = False))
model.add(tf.keras.layers.Conv2D(96,kernel_size=1,strides=[1,1], kernel_initializer=my_init_3,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same',trainable = False))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05,trainable = False))
model.add(tf.keras.layers.ReLU(trainable = False))
model.add(tf.keras.layers.ZeroPadding2D(padding=(1, 1),trainable = False))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2),trainable = False))

model.add(tf.keras.layers.Conv2D(192,kernel_size=5,strides=[1,1], kernel_initializer=my_init_1,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same',trainable = False))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05,trainable = False))
model.add(tf.keras.layers.ReLU(trainable = False))
model.add(tf.keras.layers.Conv2D(192,kernel_size=1,strides=[1,1], kernel_initializer=my_init_4,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same',trainable = False))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05,trainable = False))
model.add(tf.keras.layers.ReLU(trainable = False))
model.add(tf.keras.layers.Conv2D(192,kernel_size=1,strides=[1,1], kernel_initializer=my_init_4,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same',trainable = False))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05,trainable = False))
model.add(tf.keras.layers.ReLU(trainable = False))
model.add(tf.keras.layers.ZeroPadding2D(padding=(1, 1),trainable = False))
model.add(tf.keras.layers.AveragePooling2D(pool_size=(3, 3), strides=(2,2),trainable = False))

model.add(tf.keras.layers.Conv2D(192,kernel_size=3,strides=[1,1], kernel_initializer=my_init_5,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same'))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.Conv2D(192,kernel_size=1,strides=[1,1], kernel_initializer=my_init_4,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same'))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.Conv2D(192,kernel_size=1,strides=[1,1], kernel_initializer=my_init_4,kernel_regularizer=tf.keras.regularizers.l2(5e-4), use_bias=False, padding='same'))
model.add(tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-05))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.GlobalAveragePooling2D())

model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax,kernel_initializer=my_init_dense, kernel_regularizer=tf.keras.regularizers.l2(5e-4), bias_initializer='zeros',bias_regularizer=tf.keras.regularizers.l2(5e-4)))

my_optimizer=tf.keras.optimizers.SGD(momentum=MOMENTUM, nesterov=True)
model.compile(optimizer=my_optimizer,loss=categorical_loss,metrics=['accuracy'])
model.fit(x_train, labels, batch_size=128, epochs=100, callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
50000/50000 [==============================] - 24s 489us/sample - loss: 2.6605 - acc: 0.4999
Epoch 2/100
50000/50000 [==============================] - 23s 455us/sample - loss: 2.2803 - acc: 0.5875
Epoch 3/100
50000/50000 [==============================] - 23s 457us/sample - loss: 2.1456 - acc: 0.6184
Epoch 4/100
50000/50000 [==============================] - 23s 456us/sample - loss: 2.0816 - acc: 0.6352
Epoch 5/100
50000/50000 [==============================] - 23s 456us/sample - loss: 2.0502 - acc: 0.6483
Epoch 6/100
50000/50000 [==============================] - 23s 458us/sample - loss: 2.0284 - acc: 0.6600
Epoch 7/100
50000/50000 [==============================] - 23s 457us/sample - loss: 2.0209 - acc: 0.6662
Epoch 8/100
50000/50000 [==============================] - 23s 456us/sample - loss: 2.0140 - acc: 0.6740
Epoch 9/100
50000/50000 [==============================] - 23s 456us/sample - loss: 2.0129 - acc: 0.6782
Epoch 1

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 192)       14400     
_________________________________________________________________
batch_normalization_v1_9 (Ba (None, 32, 32, 192)       768       
_________________________________________________________________
re_lu_9 (ReLU)               (None, 32, 32, 192)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 32, 160)       30720     
_________________________________________________________________
batch_normalization_v1_10 (B (None, 32, 32, 160)       640       
_________________________________________________________________
re_lu_10 (ReLU)              (None, 32, 32, 160)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 32, 96)        15360     
__________

In [14]:
train_loss, train_acc = model.evaluate(x_train, labels)
print("loss", train_loss, "acc", train_acc)

50000/50000 [==============================] - 24s 473us/sample - loss: 6.9396 - acc: 0.1000
loss 6.939646385650635 acc 0.10004
(10000, 10)
10000/10000 [==============================] - 5s 468us/sample - loss: 6.9307 - acc: 0.1001
loss 6.9306990501403805 acc 0.1001
